In [369]:
class Board():
    loc2index = {
    "0": (0,0),
    "1": (0,1),
    "2": (0,2),
    "3": (1,0),
    "4": (1,1),
    "5": (1,2),
    "6": (2,0),
    "7": (2,1),
    "8": (2,2)
}
    index2loc = {
        "0,0": "0",
        "0,1": "1",
        "0,2": "2",
        "1,0": "3",
        "1,1": "4",
        "1,2": "5",
        "2,0": "6",
        "2,1": "7",
        "2,2": "8"
    }
    win_states = [
        set(((0,0), (0,1), (0,2))),
        set(((1,0), (1,1), (1,2))),
        set(((2,0), (2,1), (2,2))),
        set(((0,0), (1,0), (2,0))),
        set(((0,1), (1,1), (2,1))),
        set(((0,2), (1,2), (2,2))),
        set(((0,0), (1,1), (2,2))),
        set(((0,2), (1,1), (2,0)))
        ]
    
    def __init__(self):
        self.board = [[" " for _ in range(3)] for _ in range(3)]
        self.available = set([(0,0), (0,1), (0,2), (1,0), (1,1), (1,2), (2,0), (2,1), (2,2)])
        self.consumed = {
            "X": set(),
            "O": set()
        }
        self.turn_X = True
    def move(self, loc):
        x = Board.loc2index[loc][0]
        y = Board.loc2index[loc][1]
        marker = "X" if self.turn_X else "O"
        self.board[x][y] = marker
        loc_int = int(loc)
        self.available.remove((x,y))
        self.consumed[marker].add((x,y))
        self.__str__()
#         self.check_win()
        self.turn_X = not self.turn_X

    def __str__(self):
        print(" -------------")
        for row in self.board:
            print("|", " | ".join(row), "|")
            print(" -------------")
        return ""
    def check_win(self):
        for win_state in Board.win_states:
            if win_state.issubset(self.consumed["X"]):
                print("GAME OVER! Player wins!")
                return True
            elif win_state.issubset(self.consumed["O"]):
                print("GAME OVER! AI won!")
                return True
        if not self.available:
            print("GAME OVER WITH A DRAW!")
            return True
        return False

In [356]:
from collections import defaultdict
from copy import deepcopy, copy

# Minimax

In [379]:
Counter = 0
def terminal_test(available, used_X, used_O):
    for win_state in Board.win_states:
        if win_state.issubset(used_O):
            return 1
        if win_state.issubset(used_X):
            return -1
    if not available:
        return 0
    return None
    
def max_value(available, used_X, used_O):
    global Counter
    Counter+= 1
    terminal_test_result = terminal_test(available, used_X, used_O)
    if terminal_test_result is not None:
        return terminal_test_result
    v = float('-inf')
    for move in list(available):
        available.remove(move)
        used_O.add(move)
        v = max(v, min_value(available, used_X, used_O))
        available.add(move)
        used_O.remove(move)
    return v

def min_value(available, used_X, used_O):
    global Counter
    Counter += 1
    terminal_test_result = terminal_test(available, used_X, used_O)
    if terminal_test_result is not None:
        return terminal_test_result
    v = float('inf')
    for move in list(available):
        available.remove(move)
        used_X.add(move)
        v = min(v, max_value(available, used_X, used_O))
        available.add(move)
        used_X.remove(move)
    return v

In [381]:
from copy import copy,deepcopy
game = Board()
print(game)
while not game.check_win():
    if game.turn_X:
        game.move(input())
    else:
        available = deepcopy(game.available)
        used_X = deepcopy(game.consumed["X"])
        used_O = deepcopy(game.consumed["O"])
        
        count = float('-inf')
        move_min = None
        for move in list(available):
            available.remove(move)
            used_O.add(move)
            c = min_value( available, used_X, used_O)
            if c > count:
                count = c
                move_min = move
            
            available.add(move)
            used_O.remove(move)
        print(f"Searched: {Counter}")
        game.move(Board.index2loc[str(move_min[0])+","+str(move_min[1])])

 -------------
|   |   |   |
 -------------
|   |   |   |
 -------------
|   |   |   |
 -------------

4
 -------------
|   |   |   |
 -------------
|   | X |   |
 -------------
|   |   |   |
 -------------
Searched: 111008
 -------------
| O |   |   |
 -------------
|   | X |   |
 -------------
|   |   |   |
 -------------


KeyboardInterrupt: Interrupted by user

# Alpha-Beta Pruning

In [383]:
Counter = 0
def terminal_test(available, used_X, used_O):
    for win_state in Board.win_states:
        if win_state.issubset(used_O):
            return 1
        if win_state.issubset(used_X):
            return -1
    if not available:
        return 0
    return None

def alpha_beta_search(available, used_X, used_O):
    alpha = float('-inf')
    beta = float('inf')
    v = float('-inf')
    recommended_move = None
    for move in list(available):
        available.remove(move)
        used_O.add(move)
        cur_val = min_value(available, used_X, used_O, alpha, beta)
        if cur_val > v:
            v = cur_val
            recommended_move = move
        available.add(move)
        used_O.remove(move)
    return recommended_move, v
            
    
def max_value(available, used_X, used_O, alpha, beta):
    global Counter
    Counter += 1
    terminal_test_result = terminal_test(available, used_X, used_O)
    if terminal_test_result is not None:
        return terminal_test_result
    v = float('-inf')
    for move in list(available):
        available.remove(move)
        used_O.add(move)
        v = max(v, min_value(available, used_X, used_O, alpha, beta))
        available.add(move)
        used_O.remove(move)
        if v >= beta:
            return v
        alpha = max(alpha, v)
    return v

def min_value(available, used_X, used_O, alpha, beta):
    global Counter
    Counter += 1
    terminal_test_result = terminal_test(available, used_X, used_O)
    if terminal_test_result is not None:
        return terminal_test_result
    v = float('inf')
    for move in list(available):
        available.remove(move)
        used_X.add(move)
        v = min(v, max_value(available, used_X, used_O, alpha, beta))
        available.add(move)
        used_X.remove(move)
        if v <= alpha:
            return v
        beta = min(beta, v)
    return v

In [384]:
from copy import copy,deepcopy
game = Board()
print(game)
while not game.check_win():
    if game.turn_X:
        move = input()
        if Board.loc2index[move] not in game.available:
            print("Incorrect move!")
            continue
        game.move(move)
    else:
        available = deepcopy(game.available)
        used_X = deepcopy(game.consumed["X"])
        used_O = deepcopy(game.consumed["O"])
        move_min, v = alpha_beta_search(available, used_X, used_O)
#         print(move_min, v)
        game.move(Board.index2loc[str(move_min[0])+","+str(move_min[1])])
        print(f"Searched: {Counter}")

 -------------
|   |   |   |
 -------------
|   |   |   |
 -------------
|   |   |   |
 -------------

4
 -------------
|   |   |   |
 -------------
|   | X |   |
 -------------
|   |   |   |
 -------------
 -------------
| O |   |   |
 -------------
|   | X |   |
 -------------
|   |   |   |
 -------------
Searched: 7399


KeyboardInterrupt: Interrupted by user